In [32]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# This code was provided by the project material to reduce the size of the OSM Map used for testing purposes

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"

OSM_FILE = File_Dir+"dubai_abu-dhabi.osm"
SAMPLE_OSM_FILE = File_Dir+"dubai_sample_k100.osm"

print OSM_FILE
print SAMPLE_OSM_FILE

k = 100 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_OSM_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\dubai_abu-dhabi.osm
C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\dubai_sample_k100.osm


In [29]:
#Code from the course material to handle OpenStreetMap data and print all the Nodes/Elements in the OSM Map.

#osm_file = open(OSM_FILE, "r")
osm_file = open(SAMPLE_OSM_FILE, "r")
tree = ET.parse(osm_file) #parse file into XML tree
root = tree.getroot() #get the root node of the XML tree
root_children = set()
for element in root: #loop on direct children of root
    root_children.add(element.tag)
print "Root Children",root_children
all_nodes = set()
for element in root.iter(): #loop on all nodes (depth first)
    all_nodes.add(element.tag)
print "All Nodes",all_nodes

Root Children set(['node', 'relation', 'way'])
All Nodes set(['node', 'nd', 'member', 'tag', 'relation', 'way', 'osm'])


In [146]:
# This code to print various statiscs about the project files
from subprocess import check_output
import os
File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"
OSM_MAP = "dubai_abu-dhabi.osm"
OSM_DB = "MY_OSMP.db"
File_Name = File_Dir+OSM_DB


def get_size(f):
    return (float(os.path.getsize(f)) / (1024*1024))
print "\n"

#st = os.stat(File_Name)
#print st.st_size

'''
ccc = check_output("cd "+File_Dir, shell=True)
print type(ccc)
for i in ccc.split():
    print i
check_output("Dir MY_OSMP.db", shell=True)
#check_output("@echo off", shell=True)
#check_output("echo %~z1", shell=True)
'''
NODES_PATH = File_Dir+"nodes.csv"
NODE_TAGS_PATH = File_Dir+"nodes_tags.csv"
WAYS_PATH = File_Dir+"ways.csv"
WAY_NODES_PATH = File_Dir+"ways_nodes.csv"
WAY_TAGS_PATH = File_Dir+"ways_tags.csv"

print ('Size of {} Map: \t {} MB'.format(OSM_MAP, round(get_size(OSM_MAP), 2)))
print ('Size of {} Database: \t\t {} MB'.format(OSM_DB, round(get_size(OSM_DB), 2)))

print ('Size of {}: \t\t\t {} MB'.format("nodes.csv", round(get_size(NODES_PATH), 2)))
print ('Size of {}: \t\t  {} MB'.format("nodes_tags.csv", round(get_size(NODE_TAGS_PATH), 2)))
print ('Size of {}: \t\t\t  {} MB'.format("ways.csv", round(get_size(WAYS_PATH), 2)))
print ('Size of {}:\t\t\t  {} MB'.format("ways_tags.csv", round(get_size(WAY_TAGS_PATH), 2)))
print ('Size of {}: \t\t  {} MB'.format("ways_nodes.csv", round(get_size(WAY_NODES_PATH), 2)))




#print "Size of {} OSM Database %.2f MB" % OSM_FILE, get_size(OSM_FILE)

#print "Size of OSM Database %.2f MB" % get_size(SAMPLE_OSM_FILE)
#print "Size of OSM Database %.2f MB" % get_size(SAMPLE_OSM_FILE)






Size of dubai_abu-dhabi.osm Map: 	 511.32 MB
Size of MY_OSMP.db Database: 		 3.46 MB
Size of nodes.csv: 			 1.96 MB
Size of nodes_tags.csv: 		  0.12 MB
Size of ways.csv: 			  0.19 MB
Size of ways_tags.csv:			  0.26 MB
Size of ways_nodes.csv: 		  0.65 MB


In [117]:
# This code is to fix the problem wuth street names where most of the names end with "Street" but some names start
# with "Street".  
# This code will fix this problem by changing the street names that start with "Stree" and correct it so that the stree names
# end with "Street" so that they are all uniform.

import xml.etree.cElementTree as ET
import re

# Regular expression to catch a street name starting with "Street" and assign it to a group, 
# and the remaining street name to the second group
Start_with_Street_re = re.compile(r'^(\bStreet\b\s)(.+)')

def is_street_name(elem):
    #return (elem.attrib['k'] == "addr:street")
    return (elem.attrib['k'] == "addr:street") or (elem.attrib['k'] == "name:en")
    #return ((elem.attrib['k'] == "addr:street:en") or (elem.attrib['k'] == "addr:street"))
    
def correct_street_names(osmfile):  # Not used in current code, I used this for testing initially
    osm_file = open(osmfile, "r")
    street_names_corected = []
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
        #if elem.tag == "node":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    
                    m = Start_with_Street_re.search(tag.attrib['v'])         # get the abbreviated name from the string as object
                    if m:
                        street_name_1 = m.group(1)
                        street_name_2 = m.group(2)
                        #print m.group()
                        #print street_name_2, street_name_1
                        street_names_corected.append(street_name_2 +" " + street_name_1)
    print "number of incorrect street names found and corrected: ", len(street_names_corected)
    print street_names_corected
                    
    osm_file.close()
    return street_names_corected

def correct_street_name(tag):
    #street_names_corrected = "Zaki"
    street_names_corrected = tag.attrib['v'] 
    print "Function: street name to correct.....", street_names_corrected

    if is_street_name(tag):
        m = Start_with_Street_re.search(tag.attrib['v'])         # get the abbreviated name from the string as object
        if m:
            street_name_1 = m.group(1)
            street_name_2 = m.group(2)
            #print m.group()
            #print street_name_2, street_name_1
            
            street_names_corrected = street_name_2 +" " + street_name_1
            #tag.attrib['v'] = street_name_2 +" " + street_name_1
            print "Function: correct_street_name.....", street_names_corrected

        #else:
            #street_names_corrected = tag.attrib['v']
    #print "number of incorrect street names found and corrected: ", len(street_names_corected)
                    
    return street_names_corrected

'''
def test():
    #st_types = audit(OSMFILE)
    #street_names = correct_street_name(OSMFILE)
    street_names = correct_street_name(SAMPLE_OSM_FILE)
                                       
    #pprint.pprint(dict(street_names))
'''
'''
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
'''         

if __name__ == '__main__':
    street_names = correct_street_names(SAMPLE_OSM_FILE)

number of incorrect street names found and corrected:  58
['16 Street ', '28 Street ', '40 Street ', '65 Street ', '59 Street ', '9 Street ', '9 Street ', '25 Street ', '21a Street ', '20 Street ', '11 Street ', '21 Street ', '22d Street ', '2d Street ', '10b Street ', '47 Street ', '24 Street ', '2 Street ', '6 Street ', '46 Street ', '34 Street ', '26 Street ', '7 Street ', '19 Street ', '15 Street ', '2 Street ', '9 Street ', '16 Street ', '9 Street ', '8 Street ', '30 Street ', '2 Street ', '9c Street ', '6 Street ', '33 Street ', '23 Street ', '49 Street ', '23 Street ', '78 Street ', '32 Street ', '18 Street ', '30 Street ', '67 Street ', '41 Street ', '17 Street ', '59 Street ', '78 Street ', '17 Street ', '13 Street ', '22 Street ', '3 Street ', '10 Street ', '43a Street ', '91a Street ', '81b Street ', '22 Street ', '54 Street ', '42 Street ']


In [ ]:
# coding=utf-8


In [62]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# This code was taken from the OpenStreetMap Case Study Excercise and use it here to write the OSM XML data to CSV format
# I have completed this excercise and I am reusing the code here.


###import unicodecsv as csv
#import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus
import schema

File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"
OSM_PATH = File_Dir+"dubai_sample_k100.osm"

NODES_PATH = File_Dir+"nodes.csv"
NODE_TAGS_PATH = File_Dir+"nodes_tags.csv"
WAYS_PATH = File_Dir+"ways.csv"
WAY_NODES_PATH = File_Dir+"ways_nodes.csv"
WAY_TAGS_PATH = File_Dir+"ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements
    

    # YOUR CODE HERE:
    tag_key_value = None
    tag_type_value = None
    corrected_street_name = None
    

    # handle and format node elements
    if element.tag == 'node':
        node_attribs['id'] = element.attrib['id']
        node_attribs['user'] = element.attrib['user']
        node_attribs['uid'] = element.attrib['uid']
        node_attribs['version'] = element.attrib['version']
        node_attribs['lat'] = element.attrib['lat']
        node_attribs['lon'] = element.attrib['lon']
        node_attribs['timestamp'] = element.attrib['timestamp']
        node_attribs['changeset'] = element.attrib['changeset']
        
        # handle and format "tag" elements, iterate to catch all of them
        for tag in element.iter("tag"):
            if not PROBLEMCHARS.search(tag.attrib['k']):        # if it has these characters then ignore
                corrected_street_name = tag.attrib['v']
                tag_colon = LOWER_COLON.search(tag.attrib['k'])         # check if tag has a colon
                if tag_colon:
                
                    tag_key_value = tag.attrib['k'].split(":", 1)[1]    #only split once to account for more ":" in string
                    tag_type_value = tag.attrib['k'].split(":")[0]
                    
                    #tag_type_value.encode("utf-8").decode('unicode-escape')
                    ###tag_type_value.encode("utf-8")
                                        
                    # check and correct the street name if it starts with "Street"
                    #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ"
                    if tag_key_value == "street":
                        corrected_street_name = correct_street_name(tag)
                else:
                    tag_key_value = tag.attrib['k']
                    tag_type_value = "regular"

                tags.append({'id': node_attribs['id'], 'key':tag_key_value, 'value': corrected_street_name, 'type': tag_type_value})
                #tags.append({'id': node_attribs['id'], 'key':tag_key_value, 'value': tag.attrib['v'], 'type': tag_type_value})

        #print tags
        return {'node': node_attribs, 'node_tags': tags}
        
    # handle and format way elements
    elif element.tag == 'way':
        way_attribs['id'] = element.attrib['id']
        way_attribs['user'] = element.attrib['user']
        way_attribs['uid'] = element.attrib['uid']
        way_attribs['version'] = element.attrib['version']
        way_attribs['timestamp'] = element.attrib['timestamp']
        way_attribs['changeset'] = element.attrib['changeset']
        
        # handle and format "tag" elements, itgerate to catch all of them
        tags = []
        for tag in element.iter("tag"):
            if not PROBLEMCHARS.search(tag.attrib['k']):        # if it has these characters then ignore
                corrected_street_name = tag.attrib['v']
                tag_colon = LOWER_COLON.search(tag.attrib['k'])         # check if tag has a colon
                if tag_colon:
                    tag_key_value = tag.attrib['k'].split(":", 1)[1]    #only split once to account for more ":" in string
                    tag_type_value = tag.attrib['k'].split(":")[0]
                    
                    # check and correct the street name if it starts with "Street"
                    #ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ"
                    if tag_key_value == "street" or (tag_key_value == "en" and tag_type_value == "name"):
                        corrected_street_name = correct_street_name(tag)
                else:
                    tag_key_value = tag.attrib['k']
                    tag_type_value = "regular"

                #tags.append({'id': way_attribs['id'], 'key':tag_key_value, 'value': tag.attrib['v'], 'type': tag_type_value})
                tags.append({'id': way_attribs['id'], 'key':tag_key_value, 'value': corrected_street_name, 'type': tag_type_value})

        #print tags
        
        # handle and format "nd" elements, iterate to catch all of them
        nd_count = 0
        for nd in element.iter("nd"):
            way_nodes.append({'id': way_attribs['id'], 'node_id': nd.attrib['ref'], 'position': nd_count})
            nd_count += 1
                
        #print way_nodes
            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:
                
        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ###node_tags_writer = csv.writer(nodes_tags_file, NODE_TAGS_FIELDS, encoding='utf-8')

        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)
        


        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()
        

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

Function: street name to correct..... Jumeirah Beach Road
Function: street name to correct..... Al Hubob Street
Function: street name to correct..... 16th Street
Function: street name to correct..... Al Nahda (26th) Street
Function: street name to correct..... 9th Street
Function: street name to correct..... 11 Street
Function: street name to correct..... Bu Fahhal Street
Function: street name to correct..... Meaisem Street
Function: street name to correct..... sheikh khalid bin khalid street
Function: street name to correct..... Al Barsha south 2 
Function: street name to correct..... Maliha Road
Function: street name to correct..... al rawada
Function: street name to correct..... Sheikh Zayed Road
Function: street name to correct..... Al Wasl Road
Function: street name to correct..... Al Ittihad Road
Function: street name to correct..... Sheikh Rashid bin Saeed Street
Function: street name to correct..... 103 Al Sharq Street
Function: street name to correct..... Corniche Road
Functio

Function: street name to correct..... Sheikh Ammar Street
Function: street name to correct..... شارع الشيخ راشد بن حميد
Function: street name to correct..... Al Khor Street
Function: street name to correct..... Al Khor Street
Function: street name to correct..... Al Khor Street
Function: street name to correct..... Othman Bin Madhoon Street
Function: street name to correct..... Souk Deira Street
Function: street name to correct..... 12 Street
Function: street name to correct..... 14c Street
Function: street name to correct..... 14a Street
Function: street name to correct..... Sheikh Zayed Street
Function: street name to correct..... 50 Street
Function: street name to correct..... 8 Street
Function: street name to correct..... 16 Street
Function: street name to correct..... Jamal Abdul Naser Street
Function: street name to correct..... Jamal Abdul Naser Street
Function: street name to correct..... King Abdul Aziz Street
Function: street name to correct..... Al Zahra'a Street
Function: s

Function: street name to correct..... شارع جَزِيرَة غَاغَة
Function: street name to correct..... شارع جَزيِرَة الحَنْيُورَة
Function: street name to correct..... شارع جَزِيرَة غَاغَة
Function: street name to correct..... شارع هَرَامِي
Function: street name to correct..... شارع سَمْحون
Function: street name to correct..... شارع رَأس مِسَيد
Function: street name to correct..... شارع الظفرة
Function: street name to correct..... شارع بو يَرْدِية
Function: street name to correct..... شارع بو يَرْدِية
Function: street name to correct..... شارع صِلِيج
Function: street name to correct..... شارع الثِّقَيْبَة
Function: street name to correct..... شارع السَّمَاك
Function: street name to correct..... شارع نِقا لَفْهَاد
Function: street name to correct..... شارع المَاسِي
Function: street name to correct..... شارع طَوِي نَدَا
Function: street name to correct..... شارع البَكْرَا
Function: street name to correct..... شارع صَفْرَاوِيَّة
Function: street name to correct..... شارع الغُمَّيْضَة
Function: 

Function: street name to correct..... 1 Street
Function: street name to correct..... Street 26
Function: correct_street_name..... 26 Street 
Function: street name to correct..... Al Qasbah Street
Function: street name to correct..... 21c Street
Function: street name to correct..... Street 7
Function: correct_street_name..... 7 Street 
Function: street name to correct..... Ajban Road
Function: street name to correct..... 34b Street
Function: street name to correct..... Road G
Function: street name to correct..... 1st Street
Function: street name to correct..... 70th Street
Function: street name to correct..... 74th Street
Function: street name to correct..... Maitha Bint Mohammed Street
Function: street name to correct..... 35th Street
Function: street name to correct..... 27th Street
Function: street name to correct..... Street 19
Function: correct_street_name..... 19 Street 
Function: street name to correct..... 64th Street
Function: street name to correct..... 4th Street
Function: st

Function: street name to correct..... 10th Street
Function: street name to correct..... Shakhbout Bin Sultan Street
Function: street name to correct..... Sheikh Mohammed bin Zayed Road
Function: street name to correct..... Mina Al Arab
Function: street name to correct..... Baghdad Street
Function: street name to correct..... Shakespeare Street
Function: street name to correct..... Violet Place
Function: street name to correct..... Lilly Lane
Function: street name to correct..... Daisy Way
Function: street name to correct..... 2 Street
Function: street name to correct..... Elite 7
Function: street name to correct..... Al Thammam
Function: street name to correct..... 35 Street
Function: street name to correct..... G10 Street
Function: street name to correct..... Al Qudra Road
Function: street name to correct..... Fujairah Road
Function: street name to correct..... Barton Street
Function: street name to correct..... Al Tamtam Street
Function: street name to correct..... 32nd Street
Functi

Function: street name to correct..... Sheikh Zayed Road
Function: street name to correct..... 23rd Street
Function: street name to correct..... 53rd Street
Function: street name to correct..... 42a Street
Function: street name to correct..... 10th Street
Function: street name to correct..... 44c Street
Function: street name to correct..... 54 Street
Function: street name to correct..... 10th Street
Function: street name to correct..... 3 a Street
Function: street name to correct..... 1 Street
Function: street name to correct..... 3 Street
Function: street name to correct..... 1 Street
Function: street name to correct..... 6 Street
Function: street name to correct..... G3 Street
Function: street name to correct..... Bani Yas Street
Function: street name to correct..... 3a Street
Function: street name to correct..... 5th Street
Function: street name to correct..... Al Wasl Road
Function: street name to correct..... 39b Street
Function: street name to correct..... 30th Street
Function: st

Function: street name to correct..... Arwa 2
Function: street name to correct..... Al Kasir Street
Function: street name to correct..... Street 54
Function: correct_street_name..... 54 Street 
Function: street name to correct..... Street 42
Function: correct_street_name..... 42 Street 
Function: street name to correct..... Al Jeer Road


In [30]:
# code to print the street names to see if there are any problems
# Reference:  This code was taken from the OpenStreetMap Case study, which I completed online.

import xml.etree.cElementTree as ET
from collections import defaultdict
import pprint

File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"
OSMFILE = File_Dir+"dubai_sample_k100.osm"

'''
def is_street_name(elem):
    #return (elem.attrib['k'] == "addr:street")
    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ
    return ((elem.attrib['k'] == "addr:street:en") or (elem.attrib['k'] == "addr:street"))
    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ
                    print tag.attrib['v']
                    #audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types
'''

streets = set()
for element in root.iter("way"):
    for t in element.findall("tag"):
        if t.attrib['k'] == "addr:street":
            streets.add(t.attrib['v'])
print len(streets)
#print streets
pprint.pprint(streets)

129
set(['1 Street',
     '102 Street',
     '107 Street',
     '11 Street',
     '121 Street',
     '124 Street',
     '129 Street',
     '13c Street',
     '145 Street',
     '147 Street',
     '14A Street',
     '14b Street',
     '14d Street',
     '15 Street',
     '16 Street',
     '16a Street',
     '17a Street',
     '17b Street',
     '17c Street',
     '19 Street',
     '21st Street',
     '23 Street',
     '23a Street',
     '25c Street',
     '26 Street',
     '27a Street',
     '27c Street',
     '29th Street',
     '3 Street',
     '3 a Street',
     '30th Street',
     '31 Street',
     '32a Street',
     '33 Street',
     '33rd Street',
     '34b Street',
     '36 Street',
     '38a Street',
     '3b Street',
     '40th Street',
     '44b Street',
     '46 Street',
     '49e Street',
     '5 Street',
     '5 b Street',
     '53 Street',
     '53a Street',
     '54c Street',
     '6 Street',
     '63 Street',
     '65a Street',
     '70 Street',
     '74 Street',
     '7

In [33]:
# This code now only print the all the street names ............ IGNORE AND MAYBE DELETE ONCE DONE ---------------
# This code is to fix the problem wuth street names where most of the names end with "Street" but some names start
# with "Street".  
# This code will fix this problem by changing the street names that start with "Stree" and correct it so that the stree names
# end with "Street" so that they are all uniform.

"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import difflib


File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"
OSMFILE = File_Dir+"dubai_sample_k100.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    #return (elem.attrib['k'] == "addr:street")
    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ
    return ((elem.attrib['k'] == "addr:street:en") or (elem.attrib['k'] == "addr:street"))
    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ




def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    ###################################################
                    print tag.attrib['v']
                    #######################################################################
                    #audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

# I added this function to retuen the matched leng name from the "expected" list to a given abbreviation
def get_long_name(street_type):
    # if the abbreviation is cotained in one of the "expeted" list then it is a simple match
    # example "Ave" is contained in "Avenue"
    for long_name in expected:
        if street_type in long_name:
            return long_name
        # if not, then the abbreviation is not simple, such as "Rd" will not match "Road" by the above if block
        # I choose to do this fuzzy match using the difflib python library, which can find a close match
        # to the abbreviated street name (example "Rd" is a close match for "Road"
        else:
            street_type = street_type.split('.')[0]
            long_name = difflib.get_close_matches(street_type, expected)[0]
            #print street_type, long_name
            return long_name

def update_name(name, mapping):

    # YOUR CODE HERE
    street_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)     # regex to matche the street abbreviation at end of line
    #short_name = re.search(street_re, name)
    short_name = street_re.search(name)         # get the abbreviated name from the string as object
            
    if short_name:
        street_type = short_name.group()
        
        if street_type not in mapping:          # if apprevation not already in mappings, then add it
            long_name = get_long_name(street_type)
            mapping[street_type] = long_name
            #print short_name, "Added to mappings"
        else:                                   # else just get the mapping to long name from mappings
            long_name = mapping[street_type]
            
        name = re.sub(street_type, long_name, name) # change the abbreviated string to long name in full street name

    return name

def test():
    st_types = audit(OSMFILE)
    #assert len(st_types) == 3
    pprint.pprint(dict(st_types))
'''
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
'''         

if __name__ == '__main__':
    test()



Jumeirah Beach Road
Al Hubob Street
16th Street
Al Nahda (26th) Street
9th Street
11 Street
Bu Fahhal Street
Meaisem Street
sheikh khalid bin khalid street
Al Barsha south 2 
Maliha Road
al rawada
Sheikh Zayed Road
Al Wasl Road
Al Ittihad Road
Sheikh Rashid bin Saeed Street
103 Al Sharq Street
Corniche Road
Khasab Main Road
Oud Metha Road
N300
Sultan Bin Zayed The First Street
Sheikh Zayed Road
Bu Mardah Street
Street 16
2nd Street
Street 28
Al Houdhiyyat Street
Street 40
Khalifa Bin Zayed Street
'Oud Maytha Street
Al Jidaf Street
Al Nasa-im Street
Qarn Mgharraq Street
Street 65
Street 59
2nd Street
15th Street
15th Street
15th Street
Street 9
27 Street
Street 9
14th Street
Zayed The First Street
Street 25
7th Street
Diba Street
Street 21a
Nahyan Al Awwal Street
King Salman bin Abdulaziz Al Saud Street
Khalid Bin Waleed Road
11 Street
Khalid Bin Waleed Road
Umm Hurair Road
Street 20
4 Street
2 Street
2 Street
2 Street
1 Street
13 Street
38 Street
1 Street
6 Street
5 Street
Street 11
6 

Sheikh Mohammed bin Zayed Road
Mina Al Arab
Shakespeare Street
Violet Place
Lilly Lane
Daisy Way
Al Thammam
14A Street
38a Street
Hattan Street 2
Hattan Street 1
Al Rasheed Street
Qarn 'Uwyan Street
Sultan Bin Zayed The First Street
15 Street
Street 9c
Al Wasl Road
Muwaythij Street
Al Shita Street
Street 18
Street 30
Street 67
Street 41
Street 17
Street 59
Street 78
8th Street
Al Shaheen Street
Al Ghiryaf Street
Al Shaheen Street
Al Falah Drive
40th Street
Al Khuwafi Street
29th Street
Street 22
Sheikh Zayed Road
3 a Street
1 Street
3 Street
1 Street
6 Street
Al Wasl Road
30th Street
54c Street
13c Street
6 Street
Street 3
Lolow Street
17a Street
Shaikh Mohammed bin Rashid Boulevard
3b Street
Street 10
32a Street
83b Street
16 Street
Sheikh Rashid Road
53a Street
14d Street
75b Street
34b Street
121 Street
129 Street
Al Quoz 1 Street
27c Street
9 Street
33rd Street
65a Street
129 Street
44b Street
1 Street
21st Street
49e Street
14b Street
75th Street
Al Wasl Road
Camomile Lane
17b Str

In [65]:
# This code is to fix the problem wuth street names where most of the names end with "Street" but some names start
# with "Street".  
# This code will fix this problem by changing the street names that start with "Stree" and correct it so that the stree names
# end with "Street" so that they are all uniform.

# I copied thid code cell and moved it up before writing to csv files ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ
# This code os not used and can be deleted for the submission
"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
import difflib

'''
File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"
OSMFILE = File_Dir+"dubai_sample_k100.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    #return (elem.attrib['k'] == "addr:street")
    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ
    return ((elem.attrib['k'] == "addr:street:en") or (elem.attrib['k'] == "addr:street"))
    ##ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ




def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    ###################################################
                    print tag.attrib['v']
                    #######################################################################
                    #audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

# I added this function to retuen the matched leng name from the "expected" list to a given abbreviation
def get_long_name(street_type):
    # if the abbreviation is cotained in one of the "expeted" list then it is a simple match
    # example "Ave" is contained in "Avenue"
    for long_name in expected:
        if street_type in long_name:
            return long_name
        # if not, then the abbreviation is not simple, such as "Rd" will not match "Road" by the above if block
        # I choose to do this fuzzy match using the difflib python library, which can find a close match
        # to the abbreviated street name (example "Rd" is a close match for "Road"
        else:
            street_type = street_type.split('.')[0]
            long_name = difflib.get_close_matches(street_type, expected)[0]
            #print street_type, long_name
            return long_name

def update_name(name, mapping):

    # YOUR CODE HERE
    street_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)     # regex to matche the street abbreviation at end of line
    #short_name = re.search(street_re, name)
    short_name = street_re.search(name)         # get the abbreviated name from the string as object
            
    if short_name:
        street_type = short_name.group()
        
        if street_type not in mapping:          # if apprevation not already in mappings, then add it
            long_name = get_long_name(street_type)
            mapping[street_type] = long_name
            #print short_name, "Added to mappings"
        else:                                   # else just get the mapping to long name from mappings
            long_name = mapping[street_type]
            
        name = re.sub(street_type, long_name, name) # change the abbreviated string to long name in full street name

    return name

'''

Start_with_Street_re = re.compile(r'^(\bStreet\b\s)(.+)')


def is_street_name(elem):
    #return (elem.attrib['k'] == "addr:street")
    return ((elem.attrib['k'] == "addr:street:en") or (elem.attrib['k'] == "addr:street"))
    
def correct_street_name(osmfile):
    osm_file = open(osmfile, "r")
    #street_types = defaultdict(set) ### is set needed ??????????????????
    street_names_corected = []
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    
                    m = Start_with_Street_re.search(tag.attrib['v'])         # get the abbreviated name from the string as object
                    if m:
                        street_name_1 = m.group(1)
                        street_name_2 = m.group(2)
                        print m.group()
                        print street_name_2, street_name_1
                        street_names_corected.append(street_name_2 +" " + street_name_1)
    print "number of incorrect street names found and corrected: ", len(street_names_corected)
    print street_names_corected
                    
    osm_file.close()
    return street_names_corected


def test():
    #st_types = audit(OSMFILE)
    st_types = correct_street_name(OSMFILE)
    #assert len(st_types) == 3
    #pprint.pprint(dict(st_types))
'''
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
'''         

if __name__ == '__main__':
    test()



Street 16
16 Street 
Street 28
28 Street 
Street 40
40 Street 
Street 65
65 Street 
Street 59
59 Street 
Street 9
9 Street 
Street 9
9 Street 
Street 25
25 Street 
Street 21a
21a Street 
Street 20
20 Street 
Street 11
11 Street 
Street 21
21 Street 
Street 22d
22d Street 
Street 2d
2d Street 
Street 10b
10b Street 
Street 47
47 Street 
Street 9c
9c Street 
Street 18
18 Street 
Street 30
30 Street 
Street 67
67 Street 
Street 41
41 Street 
Street 17
17 Street 
Street 59
59 Street 
Street 78
78 Street 
Street 22
22 Street 
Street 3
3 Street 
Street 10
10 Street 
Street 43a
43a Street 
Street 91a
91a Street 
Street 81b
81b Street 
Street 22
22 Street 
number of incorrect street names found and corrected:  31
['16 Street ', '28 Street ', '40 Street ', '65 Street ', '59 Street ', '9 Street ', '9 Street ', '25 Street ', '21a Street ', '20 Street ', '11 Street ', '21 Street ', '22d Street ', '2d Street ', '10b Street ', '47 Street ', '9c Street ', '18 Street ', '30 Street ', '67 Street ', '41

In [26]:
# -*- coding: utf-8 -*-
# code to write the generated csv files into sqlite database
import sqlite3
import csv
import pandas
import codecs


DB_NAME = "MY_OSMP.db"

NODES_TABLE = "nodes"
NODES_TAGS_TABLE = "nodes_tags"
WAYS_TABLE = "ways"
WAYS_TAGS_TABLE = "ways_tags"
WAYS_NODES_TABLE = "ways_nodes"


f=open(NODES_PATH,'r')           # open the csv data file
NODES_PATH.decode('utf-8')

next(f, None)                    # skip the header row
reader = csv.reader(f)
#reader = csv.reader(open(filecsv.txt, "rb"))

            

sql_conn = sqlite3.connect( DB_NAME )
sql_conn.text_factory = str  #bugger 8-bit bytestrings
cur = sql_conn.cursor()

# delete the table in case it already exists, because if it does this caused problems for me when rerunning the code
cur.execute('''DROP TABLE if exists nodes''')
cur.execute('''DROP TABLE if exists nodes_tags''')
cur.execute('''DROP TABLE if exists ways''')
cur.execute('''DROP TABLE if exists ways_tags''')
cur.execute('''DROP TABLE if exists ways_nodes''')

#cur.executescript('drop table if exists node;')    


# not needed when using Pandas code below!!!
# Code to create the required tables as per the required schemas
sql_conn.execute("""CREATE TABLE IF NOT EXISTS nodes 
                 (id INTEGER PRIMARY KEY NOT NULL, 
                  lat REAL, 
                  lon REAL, 
                  user TEXT, 
                  uid INTEGER, 
                  version INTEGER, 
                  changeset INTEGER, 
                  timestamp TEXT);""")

sql_conn.execute("""CREATE TABLE nodes_tags (
                    id INTEGER,
                    key TEXT,
                    value TEXT,
                    type TEXT,
                    FOREIGN KEY (id) REFERENCES nodes(id)
                );""")

sql_conn.execute("""CREATE TABLE ways (
                    id INTEGER PRIMARY KEY NOT NULL,
                    user TEXT,
                    uid INTEGER,
                    version TEXT,
                    changeset INTEGER,
                    timestamp TEXT
                );""")

sql_conn.execute("""CREATE TABLE ways_tags (
                    id INTEGER NOT NULL,
                    key TEXT NOT NULL,
                    value TEXT NOT NULL,
                    type TEXT,
                    FOREIGN KEY (id) REFERENCES ways(id)
                );""")

sql_conn.execute("""CREATE TABLE ways_nodes (
                    id INTEGER NOT NULL,
                    node_id INTEGER NOT NULL,
                    position INTEGER NOT NULL,
                    FOREIGN KEY (id) REFERENCES ways(id),
                    FOREIGN KEY (node_id) REFERENCES nodes(id)
                );""")
                    

'''
# Insert data into node table using normal python
for f1, f2, f3, f4, f5, f6, f7, f8 in reader:
    cur.execute("""INSERT INTO node 
                (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?)""", 
                (f1, f2, f3, f4, f5, f6, f7, f8))
'''  

# Insert the csv generated data into the appropriate database tables using Pandas
# does the same as above code but using Pandas             
df = pandas.read_csv(NODES_PATH)
df.to_sql(NODES_TABLE, sql_conn, if_exists='append', index=False)
    
df = pandas.read_csv(NODE_TAGS_PATH)
df.to_sql(NODES_TAGS_TABLE, sql_conn, if_exists='append', index=False)

df = pandas.read_csv(WAYS_PATH)
df.to_sql(WAYS_TABLE, sql_conn, if_exists='append', index=False)

df = pandas.read_csv(WAY_TAGS_PATH)
df.to_sql(WAYS_TAGS_TABLE, sql_conn, if_exists='append', index=False)

df = pandas.read_csv(WAY_NODES_PATH)
df.to_sql(WAYS_NODES_TABLE, sql_conn, if_exists='append', index=False)
  
f.close()
sql_conn.commit()

cur.close()
sql_conn.close()

In [22]:
# this code is not printing results and stats


import sqlite3

File_Dir = "C:\Users\Zaki\Documents\Data Analyst Nanodegree - Udacity\OpenStreetMap Project\\"
DB_NAME = File_Dir+"MY_OSMP.db"

# open and connect to the Database
sql_conn = sqlite3.connect(DB_NAME)
sql_conn.text_factory = str  #bugger 8-bit bytestrings
cur = sql_conn.cursor()


###sql_conn.execute('SELECT * FROM node;') 

#r=sql_conn.fetchone()
#print sql_conn.description

# get the number of nodes in the Osm Database
#sql_conn.execute('SELECT * FROM node;') 
#sql_conn.execute("PRAGMA table_info(node);")

#results = cur.fetchall()
#print(results) 

#This works and prints
res = sql_conn.execute("PRAGMA table_info(node);")
for row in res.fetchall():
    print row[0], row[1]

print "======================"
#first way to print
res = sql_conn.execute("SELECT COUNT(*) FROM node;")
for row in res.fetchall():
    print row[0]

print "======================"
# Second way to print queries
#sql_conn.execute('SELECT COUNT(*) FROM node;') 
cur.execute("SELECT count(*) FROM node;")
#results = cur.fetchall()
#print(results)
#print(cur.fetchall())
rows = cur.fetchall()
for row in rows:
    print row
    
print "======================"
# Third way to print
for row in cur.execute('SELECT count(*) FROM node'):
    print row
          
cur.close()
sql_conn.close()

0 id
1 lat
2 lon
3 user
4 uid
5 version
6 changeset
7 timestamp
24703
(24703,)
(24703,)
